# Instance - IRIS Operations Agent: Complete Workflow Demo

This notebook demonstrates the full workflow of the Instance system:
1. Error Generation (User Story 1)
2. Message Output (User Story 2)
3. Command Reception (User Story 3)
4. Orchestration & Execution (User Stories 4-6)

## Setup

In [ ]:
# Configure environment
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

# Configure logging
from src.utils.logger import configure_logging, get_logger
configure_logging(log_level="INFO", log_format="console")
logger = get_logger("demo")

print(f"✓ Project root: {project_root}")
print(f"✓ Logging configured")

## Demo 1: Error Generation

Generate realistic IRIS error messages based on actual log patterns.

In [ ]:
from src.agents.error_generator import ErrorGeneratorAgent
from src.models.error_message import ErrorGenerationRequest

# Initialize agent with sample log
log_path = project_root / "tests" / "fixtures" / "sample_messages.log"
agent = ErrorGeneratorAgent(log_samples_path=log_path)

# Generate a license error
request = ErrorGenerationRequest(
    error_category="license",
    severity=2
)

error = agent.generate_error(request)

print("Generated Error Message:")
print("=" * 80)
print(error.to_log_format())
print("\nDetails:")
print(f"  Timestamp: {error.timestamp}")
print(f"  Process ID: {error.process_id}")
print(f"  Severity: {error.severity} (0=info, 1=warning, 2=error, 3=fatal)")
print(f"  Category: {error.category}")
print(f"  Message: {error.message_text}")

## Demo 2: Multiple Error Categories

Generate errors across all supported categories.

In [ ]:
categories = ["config", "license", "os", "journal"]

print("Error Examples by Category:")
print("=" * 80)

for category in categories:
    request = ErrorGenerationRequest(
        error_category=category,
        severity=2
    )
    error = agent.generate_error(request)
    print(f"\n[{category.upper()}]")
    print(error.to_log_format())

## Demo 3: Message Sender Service

Send generated errors to external monitoring systems.

In [ ]:
from src.services.message_sender import MessageSenderService

# Initialize sender (endpoint will queue messages if unavailable)
sender = MessageSenderService(output_endpoint="http://localhost:8080/api/messages")

# Send error message
error = agent.generate_error(ErrorGenerationRequest(error_category="license", severity=2))
result = sender.send_message(error)

print("Message Transmission Result:")
print(f"  Status: {result['status']}")
print(f"  Queued: {result.get('queued', False)}")
print(f"\nQueue Size: {sender.get_queue_size()} messages")

## Demo 4: Command Reception

Receive and validate remediation commands from external systems.

In [ ]:
from src.services.command_receiver import CommandReceiverService
from src.models.remediation_command import RemediationCommand

# Initialize receiver
receiver = CommandReceiverService()

# Create sample commands
commands = [
    RemediationCommand(
        action_type="config_change",
        target="iris.cpf",
        parameters={"section": "Startup", "key": "globals", "value": "20000"},
        priority="high"
    ),
    RemediationCommand(
        action_type="os_reconfig",
        target="hugepages",
        parameters={"resource_type": "memory", "target_value": 16384},
        priority="medium"
    ),
    RemediationCommand(
        action_type="restart",
        target="instance",
        parameters={"mode": "graceful", "timeout_seconds": 60},
        priority="critical"
    )
]

# Receive commands
print("Receiving Commands:")
print("=" * 80)
for cmd in commands:
    result = receiver.receive_command(cmd.model_dump_json())
    print(f"\n{cmd.action_type.upper()}:")
    print(f"  Status: {result['status']}")
    print(f"  Queue Position: {result['queue_position']}")
    print(f"  Priority: {cmd.priority}")

# Show queue status
status = receiver.get_queue_status()
print(f"\nQueue Status:")
print(f"  Total Commands: {status['total_commands']}")
print(f"  By Priority: {status['by_priority']}")
print(f"  By Action: {status['by_action']}")

## Demo 5: Orchestrator Agent

Route commands to appropriate specialized agents.

In [ ]:
from src.agents.orchestrator import OrchestratorAgent

# Initialize orchestrator
orchestrator = OrchestratorAgent()

# Route commands
print("Orchestrator Routing Decisions:")
print("=" * 80)

for cmd in commands:
    response = orchestrator.route_command(cmd)
    print(f"\n{cmd.action_type.upper()} → {response.agent_type.upper()} Agent")
    print(f"  Rationale: {response.rationale}")
    print(f"  Risk: {response.estimated_risk}")
    print(f"  Requires Validation: {response.requires_validation}")

## Demo 6: Config Agent

Execute configuration changes with backup and rollback.

In [ ]:
from src.agents.config_agent import ConfigAgent

# Initialize with test CPF file
test_cpf = project_root / "tests" / "fixtures" / "sample_iris.cpf"
config_agent = ConfigAgent(cpf_path=test_cpf)

# Execute config change
response = config_agent.modify_config(
    section="Startup",
    key="globals",
    value="20000"
)

print("Configuration Change Result:")
print("=" * 80)
print(f"  Success: {response.success}")
print(f"  Section: {response.section}")
print(f"  Key: {response.key}")
print(f"  Old Value: {response.old_value}")
print(f"  New Value: {response.new_value}")
print(f"  Requires Restart: {response.requires_restart}")
print(f"  Backup: {response.backup_path}")

## Demo 7: Cost Tracking

View token usage and cost statistics.

In [ ]:
from src.utils.cost_tracker import get_cost_tracker

tracker = get_cost_tracker()
stats = tracker.get_stats()

print("Token Usage Statistics:")
print("=" * 80)
print(f"  Total Operations: {stats['total_operations']}")
print(f"  Total Cost: ${stats['total_cost_usd']:.4f}")
print(f"\nWorkflow Budget:")
print(f"  Used: {stats['workflow_used']:,} / {stats['workflow_limit']:,} tokens")
print(f"  Remaining: {stats['workflow_limit'] - stats['workflow_used']:,} tokens")
print(f"\nSession Budget:")
print(f"  Used: {stats['session_used']:,} / {stats['session_limit']:,} tokens")
print(f"\nDaily Budget:")
print(f"  Used: {stats['daily_used']:,} / {stats['daily_limit']:,} tokens")

## Summary

This demo has shown:
1. ✅ Error generation for all categories (config, license, os, journal)
2. ✅ Message transmission to external systems with queueing
3. ✅ Command reception and validation with priority ordering
4. ✅ Intelligent orchestration and agent routing
5. ✅ Configuration changes with backup/rollback
6. ✅ Token usage tracking and cost management

### Next Steps
- Explore individual agent notebooks in `notebooks/demos/`
- Review implementation in `src/agents/`
- Run tests with `pytest tests/`
- Check documentation in `specs/001-iris-ops-agent/`